In [1]:
from premise import *
import bw2data

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
ndb = NewDatabase(
    scenarios=[
        {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2050},
        #{"model":"image", "pathway":"SSP2-RCP26", "year":2050},
        #{"model":"image", "pathway":"SSP2-Base", "year":2035},
        #{"model":"image", "pathway":"SSP2-Base", "year":2050},
        #{"model":"image", "pathway":"SSP2-Base", "year":2030},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="ecoinvent-3.10-biosphere",
    keep_imports_uncertainty=True
)

premise v.(2, 1, 8)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.update(
    [
        "renewables",
        #"electricity"
    ]
)

Processing scenarios for 1 sectors:   0%|       | 0/1 [00:35<?, ?it/s]

+--------------+------------+-----------------------+---------------------+
| Turbine type | Power (kW) |       Component       |    Scaling factor   |
+--------------+------------+-----------------------+---------------------+
|   offshore   |    1000    |       foundation      | 0.04209721772889033 |
|   offshore   |    1000    |         tower         |  0.4255778633310964 |
|   offshore   |    1000    |        platform       |         0.0         |
|   offshore   |    1000    |     grid connector    |  2.558398220244716  |
|   offshore   |    1000    |        nacelle        | 0.10786887180142117 |
|   offshore   |    1000    |         rotor         | 0.17900358277563716 |
|   offshore   |    1000    |         other         |         0.0         |
|   offshore   |    1000    | transformer + cabinet |         0.0         |
|   offshore   |    2000    |       foundation      | 0.09576108055645423 |
|   offshore   |    2000    |         tower         |  0.7550646611533551 |
|   offshore

AssertionError: 

In [5]:
ndb.write_db_to_brightway("wind turbine test 22")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Title: Writing activities to SQLite3 database:
  Started: 01/12/2025 18:48:58
  Finished: 01/12/2025 18:49:19
  Total time elapsed: 00:00:20
  CPU %: 91.70
  Memory %: 14.88
Created database: wind turbine test 22
Generate scenario report.


KeyError: 'region'

In [35]:
import numpy as np
from scipy.stats import truncnorm
from scipy.optimize import minimize

def get_fleet_distribution(power: int) -> dict:
    """
    Generate a distribution of wind turbine capacities such that
    the weighted average equals the fleet's average capacity.

    Parameters:
    - power (int): Fleet average capacity in kW.

    Returns:
    - dict: Distribution of turbine capacities (bin centers) with percentages.
    """

    # Parameters for the log-normal distribution
    sigma = 0.7  # Standard deviation (controls skewness)
    min_capacity = 1000  # Minimum capacity (kW)
    max_capacity = 25000  # Maximum capacity (kW)

    # Define bin edges and centers
    bin_edges = np.arange(min_capacity, max_capacity + 1000, 1000)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Generate truncated log-normal data
    mu = np.log(power) - (sigma ** 2 / 2)
    a, b = (np.log(min_capacity) - mu) / sigma, (np.log(max_capacity) - mu) / sigma
    n_samples = 100000
    lognormal_samples = truncnorm.rvs(a, b, loc=mu, scale=sigma, size=n_samples)
    capacities = np.exp(lognormal_samples)

    # Compute histogram counts
    counts, _ = np.histogram(capacities, bins=bin_edges)

    # Initial raw distribution
    raw_distribution = counts / counts.sum()

    # Optimization target: match the weighted average to the target
    def objective(weights):
        return np.sum((weights / weights.sum()) * bin_centers) - power

    # Constraint: weights must sum to 1
    constraints = {"type": "eq", "fun": lambda w: w.sum() - 1}

    # Bounds: weights must be non-negative
    bounds = [(0, 1) for _ in range(len(bin_centers))]

    # Optimize weights
    result = minimize(
        lambda w: abs(objective(w)),  # Minimize the absolute difference
        x0=raw_distribution,  # Initial guess
        bounds=bounds,
        constraints=constraints,
        method="SLSQP"  # Sequential Least Squares Programming
    )

    if not result.success:
        raise ValueError("Optimization failed to converge.")

    # Extract optimized weights
    optimized_weights = result.x / result.x.sum()

    # Validate the weighted average
    validated_average = np.sum(optimized_weights * bin_centers)
    print(f"Validated Average: {validated_average:.2f} kW (Target: {power} kW)")

    # Return the final distribution as a dictionary
    return {int(center): optimized_weights[i] for i, center in enumerate(bin_centers)}

r = get_fleet_distribution(22000)

Validated Average: 22000.00 kW (Target: 22000 kW)


In [12]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 16.6 MB/s eta 0:00:00a 0:00:01


In [ ]:
from pycoast import ContourWriter

ShapefileException: Unable to open gshhs/GSHHS_shp/l/GSHHS_l_L1.dbf or gshhs/GSHHS_shp/l/GSHHS_l_L1.shp.